In [2]:
import pandas as pd

In [ ]:
lol_df = pd.read_csv("LoL_Data.csv")
display(lol_df)

In [ ]:
filter_lol_df = lol_df.loc[lol_df["champion"] == "Aatrox"]
display(filter_lol_df)


Criar nova coluna Genero:

In [ ]:
# lol_df.loc[:, "gender"] = ''
display(lol_df)

In [ ]:
lol_df.loc[lol_df["champion"] == "Aatrox", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Ahri", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Akali", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Akshan", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Alistar", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Amumu", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Anivia", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Annie", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Aphelios", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Ashe", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Aurelion_Sol", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Azir", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Bard", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Bel'Veth", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Blitzcrank", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Brand", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Braum", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Caitlyn", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Camille", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Cassiopeia", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Cho'Gath", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Corki", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Darius", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Diana", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Dr._Mundo", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Draven", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Ekko", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Elise", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Evelynn", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Ezreal", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Fiddlesticks", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Fiora", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Fizz", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Galio", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Gangplank", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Garen", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Gnar", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Gragas", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Graves", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Gwen", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Hecarim", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Heimerdinger", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Illaoi", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Irelia", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Ivern", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Janna", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Jarvan_IV", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Jax", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Jayce", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Jhin", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Jinx", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Kai'Sa", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Kalista", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Karma", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Karthus", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Kassadin", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Katarina", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Kayle", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Kayn", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Kennen", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Kha'Zix", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Kindred", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Kled", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Kog'Maw", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "LeBlanc", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Lee_Sin", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Leona", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Lillia", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Lissandra", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Lucian", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Lulu", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Lux", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Malphite", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Malzahar", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Maokai", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Master_Yi", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Miss_Fortune", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Mordekaiser", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Morgana", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Nami", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Nasus", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Nautilus", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Neeko", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Nidalee", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Nocturne", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Nunu", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Olaf", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Orianna", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Ornn", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Pantheon", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Poppy", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Pyke", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Qiyana", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Quinn", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Rakan", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Rammus", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Rek'Sai", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Rell", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Renata_Glasc", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Renekton", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Rengar", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Riven", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Rumble", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Ryze", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Samira", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Sejuani", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Senna", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Seraphine", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Sett", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Shaco", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Shen", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Shyvana", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Singed", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Sion", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Sivir", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Skarner", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Sona", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Soraka", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Swain", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Sylas", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Syndra", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Tahm_Kench", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Taliyah", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Talon", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Taric", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Teemo", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Thresh", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Tristana", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Trundle", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Tryndamere", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Twisted_Fate", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Twitch", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Udyr", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Urgot", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Varus", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Vayne", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Veigar", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Vel'Koz", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Vex", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Vi", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Viego", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Viktor", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Vladimir", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Volibear", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Warwick", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Wukong", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Xayah", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Xerath", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Xin_Zhao", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Yasuo", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Yone", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Yorick", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Yuumi", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Zac", "gender"] = "Other"
lol_df.loc[lol_df["champion"] == "Zed", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Zeri", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Ziggs", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Zilean", "gender"] = "Male"
lol_df.loc[lol_df["champion"] == "Zoe", "gender"] = "Female"
lol_df.loc[lol_df["champion"] == "Zyra", "gender"] = "Female"
display(lol_df)